# 🌊 Wave Toolkit - Project Book

> **Interactive notebook system for the Wave Toolkit and SpiralSafe ecosystem**

[![SpiralSafe](https://img.shields.io/badge/🌀_SpiralSafe-Ecosystem-purple?style=flat-square)](https://github.com/toolate28/SpiralSafe)
[![Wave Toolkit](https://img.shields.io/badge/🌊_Wave_Toolkit-Main-0066FF?style=flat-square)](README.md)

This notebook provides an interactive environment for working with the Wave Toolkit. It includes:

- **Core Features**: Context capture, prompt generation, session workflow
- **Framework Tools**: Extensions for SpiralSafe ecosystem integration
- **Interactive Exploration**: Work with Wave patterns in real-time

---

## 📦 Setup & Imports

Initialize the notebook environment with required libraries.

In [ ]:
import json
import os
import platform
import subprocess
import shutil
from datetime import datetime
from pathlib import Path
from typing import Optional, Dict, Any, List
from dataclasses import dataclass, asdict, field

# Display version info
print(f"🌊 Wave Toolkit - Project Book")
print(f"Python: {platform.python_version()}")
print(f"Platform: {platform.system()} {platform.release()}")
print(f"Timestamp: {datetime.now().isoformat()}")

---

## 🔧 Part I: Core Wave Features

Python implementations of the core Wave toolkit functionality.

### 1.1 Context Capture

Dynamically captures your current environment context - equivalent to `Get-WaveContext.ps1`.

In [ ]:
@dataclass
class MachineContext:
    """Machine identity information."""
    name: str
    arch: str
    os: str
    cores: int

@dataclass
class UserContext:
    """User context information."""
    name: str
    home: str
    domain: Optional[str] = None

@dataclass
class ShellContext:
    """Shell environment information."""
    name: str
    version: str
    environment: str

@dataclass
class SessionContext:
    """Current working session context."""
    cwd: str
    is_git_repo: bool
    git_branch: Optional[str] = None

@dataclass
class ToolsContext:
    """Available tools detection."""
    git: bool
    node: bool
    python: bool
    docker: bool
    claude: bool
    jupyter: bool

@dataclass
class WaveContext:
    """Complete Wave environment context."""
    timestamp: str
    machine: MachineContext
    user: UserContext
    shell: ShellContext
    session: SessionContext
    tools: ToolsContext
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert to dictionary for JSON serialization."""
        return asdict(self)
    
    def to_json(self, indent: int = 2) -> str:
        """Convert to JSON string."""
        return json.dumps(self.to_dict(), indent=indent)


def check_command_exists(cmd: str) -> bool:
    """Check if a command exists in the system PATH."""
    return shutil.which(cmd) is not None


def get_git_branch() -> Optional[str]:
    """Get current git branch if in a git repository."""
    try:
        result = subprocess.run(
            ["git", "branch", "--show-current"],
            capture_output=True,
            text=True,
            timeout=5
        )
        if result.returncode == 0:
            return result.stdout.strip() or None
    except (subprocess.TimeoutExpired, FileNotFoundError, subprocess.SubprocessError):
        pass
    return None


def is_git_repo(path: str = ".") -> bool:
    """Check if the given path is inside a git repository."""
    return Path(path, ".git").exists() or Path(path).joinpath(".git").exists()


def get_wave_context() -> WaveContext:
    """Capture the current Wave environment context."""
    cwd = os.getcwd()
    is_repo = is_git_repo(cwd)
    
    return WaveContext(
        timestamp=datetime.now().isoformat(),
        machine=MachineContext(
            name=platform.node(),
            arch=platform.machine(),
            os=f"{platform.system()} {platform.release()}",
            cores=os.cpu_count() or 1
        ),
        user=UserContext(
            name=os.getenv("USER") or os.getenv("USERNAME") or "unknown",
            home=str(Path.home()),
            domain=os.getenv("USERDOMAIN")
        ),
        shell=ShellContext(
            name="Python/Jupyter",
            version=platform.python_version(),
            environment="notebook"
        ),
        session=SessionContext(
            cwd=cwd,
            is_git_repo=is_repo,
            git_branch=get_git_branch() if is_repo else None
        ),
        tools=ToolsContext(
            git=check_command_exists("git"),
            node=check_command_exists("node"),
            python=check_command_exists("python") or check_command_exists("python3"),
            docker=check_command_exists("docker"),
            claude=check_command_exists("claude"),
            jupyter=check_command_exists("jupyter")
        )
    )


# Capture and display context
context = get_wave_context()
print("📍 Wave Context Captured")
print(f"  Machine: {context.machine.name} ({context.machine.arch})")
print(f"  OS: {context.machine.os}")
print(f"  User: {context.user.name}")
print(f"  CWD: {context.session.cwd}")
if context.session.is_git_repo:
    print(f"  Git Branch: {context.session.git_branch}")

### 1.2 System Prompt Generator

Generates a context-aware system prompt - equivalent to `New-ClaudeSystemPrompt.ps1`.

In [ ]:
def generate_system_prompt(ctx: WaveContext) -> str:
    """
    Generate a Claude system prompt based on the current Wave context.
    
    Args:
        ctx: WaveContext object containing environment information
        
    Returns:
        Formatted system prompt string
    """
    # Build tools list
    tools_list = []
    if ctx.tools.git:
        tools_list.append("git")
    if ctx.tools.node:
        tools_list.append("node/npm")
    if ctx.tools.python:
        tools_list.append("python")
    if ctx.tools.docker:
        tools_list.append("docker")
    if ctx.tools.jupyter:
        tools_list.append("jupyter")
    
    tools_str = ", ".join(tools_list) if tools_list else "none detected"
    
    # Git status
    git_status = f"Yes (branch: {ctx.session.git_branch})" if ctx.session.is_git_repo else "No"
    
    # User info
    user_info = f"{ctx.user.domain}\\{ctx.user.name}" if ctx.user.domain else ctx.user.name
    
    prompt = f"""# Claude System Context

You are Claude, running in an interactive Jupyter notebook within the Wave Toolkit ecosystem.

## Environment (Auto-Detected)
- **Machine:** {ctx.machine.name} ({ctx.machine.arch}, {ctx.machine.cores} cores)
- **OS:** {ctx.machine.os}
- **User:** {user_info}
- **Environment:** {ctx.shell.name} {ctx.shell.version} ({ctx.shell.environment})
- **Working Directory:** {ctx.session.cwd}
- **Git Repo:** {git_status}
- **Available Tools:** {tools_str}

## Operating Principles
1. Provide concrete, executable code cells for Python/Jupyter
2. Prefer idempotent operations (safe to re-run)
3. Include verification steps after actions
4. State assumptions explicitly
5. When uncertain, ask rather than guess

## Collaboration Style
- Think out loud - share reasoning
- Offer alternatives when multiple approaches exist
- Build on context from earlier in the session
- Trust flows both ways

*Context generated: {ctx.timestamp}*
"""
    return prompt


# Generate and display system prompt
system_prompt = generate_system_prompt(context)
print(system_prompt)

### 1.3 Session Management

Tools for managing Wave sessions and logging - equivalent to `Invoke-ClaudeSession.ps1`.

In [ ]:
@dataclass
class WaveSession:
    """Represents a Wave collaboration session."""
    session_id: str
    timestamp: str
    context: WaveContext
    system_prompt: str
    task: Optional[str] = None
    log_entries: List[Dict[str, Any]] = field(default_factory=list)
    
    def add_log(self, entry_type: str, content: str):
        """Add a log entry to the session."""
        self.log_entries.append({
            "timestamp": datetime.now().isoformat(),
            "type": entry_type,
            "content": content
        })
    
    def save(self, output_dir: str = ".claude/logs/sessions"):
        """Save the session log to a file."""
        output_path = Path(output_dir)
        output_path.mkdir(parents=True, exist_ok=True)
        
        log_file = output_path / f"session_{self.session_id}.json"
        
        session_data = {
            "session_id": self.session_id,
            "timestamp": self.timestamp,
            "task": self.task,
            "context": self.context.to_dict(),
            "system_prompt": self.system_prompt,
            "log_entries": self.log_entries
        }
        
        with open(log_file, "w", encoding="utf-8") as f:
            json.dump(session_data, f, indent=2)
        
        return str(log_file)


def create_wave_session(task: Optional[str] = None) -> WaveSession:
    """
    Create a new Wave session with captured context.
    
    Args:
        task: Optional task description for the session
        
    Returns:
        WaveSession object
    """
    ctx = get_wave_context()
    session_id = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    session = WaveSession(
        session_id=session_id,
        timestamp=ctx.timestamp,
        context=ctx,
        system_prompt=generate_system_prompt(ctx),
        task=task
    )
    
    session.add_log("session_start", f"Session created: {session_id}")
    if task:
        session.add_log("task", task)
    
    return session


# Create a demo session
demo_session = create_wave_session("Explore Wave Toolkit Project Book")
print(f"📝 Session Created: {demo_session.session_id}")
print(f"   Task: {demo_session.task}")
print(f"   Log Entries: {len(demo_session.log_entries)}")

In [ ]:
def import_traces(json_file_path: str) -> List[Dict[str, Any]]:    """    Import trace data from a JSON file with proper error handling.        Handles JSON files containing trace data with required fields:    - trace_id: Unique identifier for the trace    - state: Current state of the trace    - input: Input data for the trace    - output: Output data from the trace        Args:        json_file_path: Path to the JSON file containing trace data            Returns:        List of trace dictionaries with validated fields            Raises:        FileNotFoundError: If the JSON file doesn't exist        json.JSONDecodeError: If the file contains invalid JSON        ValueError: If required fields are missing or invalid            Example:        >>> traces = import_traces("traces.json")        >>> for trace in traces:        ...     print(f"Trace {trace['trace_id']}: {trace['state']}")    """    # Check if file exists    if not Path(json_file_path).exists():        raise FileNotFoundError(f"Trace file not found: {json_file_path}")        # Read and parse JSON    try:        with open(json_file_path, 'r', encoding='utf-8') as f:            data = json.load(f)    except json.JSONDecodeError as e:        raise json.JSONDecodeError(            f"Invalid JSON in file {json_file_path}: {str(e)}",             e.doc,             e.pos        )        # Ensure data is a list    if isinstance(data, dict):        # If it's a single trace object, wrap it in a list        data = [data]    elif not isinstance(data, list):        raise ValueError(            f"Expected JSON to contain a list or dict, got {type(data).__name__}"        )        # Required fields for trace data    required_fields = ['trace_id', 'state', 'input', 'output']        # Validate each trace    validated_traces = []    errors = []        for idx, trace in enumerate(data):        if not isinstance(trace, dict):            errors.append(f"Trace at index {idx} is not a dictionary: {type(trace).__name__}")            continue                # Check for missing required fields        missing_fields = [field for field in required_fields if field not in trace]                if missing_fields:            # Build a helpful error message            error_msg = (                f"Trace at index {idx} is missing required fields: {', '.join(missing_fields)}. "                f"Available fields: {', '.join(trace.keys()) if trace.keys() else 'none'}. "                f"Required fields are: {', '.join(required_fields)}"            )            errors.append(error_msg)            continue                # Validate that required fields are not None        none_fields = [field for field in required_fields if trace[field] is None]        if none_fields:            error_msg = (                f"Trace at index {idx} has null values for required fields: {', '.join(none_fields)}"            )            errors.append(error_msg)            continue                validated_traces.append(trace)        # If we have errors, raise a comprehensive error message    if errors:        error_summary = f"Found {len(errors)} invalid trace(s) in {json_file_path}:\n"        error_summary += "\n".join(f"  - {err}" for err in errors[:5])  # Show first 5 errors        if len(errors) > 5:            error_summary += f"\n  ... and {len(errors) - 5} more error(s)"        raise ValueError(error_summary)        if not validated_traces:        raise ValueError(f"No valid traces found in {json_file_path}")        return validated_traces# Example usage and testdef _test_import_traces():    """Test the import_traces function with various scenarios."""    print("\n🧪 Testing import_traces function...")        # Create test directory    test_dir = Path(".claude/test_traces")    test_dir.mkdir(parents=True, exist_ok=True)        # Test 1: Valid trace data    valid_trace_file = test_dir / "valid_traces.json"    valid_data = [        {            "trace_id": "trace_001",            "state": "completed",            "input": {"query": "Hello"},            "output": {"response": "Hi there!"}        },        {            "trace_id": "trace_002",             "state": "pending",            "input": {"query": "How are you?"},            "output": {}        }    ]    with open(valid_trace_file, 'w') as f:        json.dump(valid_data, f, indent=2)        try:        traces = import_traces(str(valid_trace_file))        print(f"✅ Test 1 passed: Loaded {len(traces)} valid traces")    except Exception as e:        print(f"❌ Test 1 failed: {e}")        # Test 2: Missing required field    invalid_trace_file = test_dir / "invalid_traces.json"    invalid_data = [        {            "trace_id": "trace_003",            "state": "completed",            # Missing 'input' and 'output'        }    ]    with open(invalid_trace_file, 'w') as f:        json.dump(invalid_data, f, indent=2)        try:        traces = import_traces(str(invalid_trace_file))        print(f"❌ Test 2 failed: Should have raised ValueError for missing fields")    except ValueError as e:        if "missing required fields" in str(e):            print(f"✅ Test 2 passed: Caught missing fields error")        else:            print(f"❌ Test 2 failed: Wrong error message: {e}")    except Exception as e:        print(f"❌ Test 2 failed: Unexpected error: {e}")        # Test 3: File not found    try:        traces = import_traces("nonexistent_file.json")        print(f"❌ Test 3 failed: Should have raised FileNotFoundError")    except FileNotFoundError:        print(f"✅ Test 3 passed: Caught file not found error")    except Exception as e:        print(f"❌ Test 3 failed: Unexpected error: {e}")        # Test 4: Invalid JSON    malformed_file = test_dir / "malformed.json"    with open(malformed_file, 'w') as f:        f.write("{invalid json content")        try:        traces = import_traces(str(malformed_file))        print(f"❌ Test 4 failed: Should have raised JSONDecodeError")    except json.JSONDecodeError:        print(f"✅ Test 4 passed: Caught invalid JSON error")    except Exception as e:        print(f"❌ Test 4 failed: Unexpected error: {e}")        print("\n✨ Testing complete!")# Uncomment to run tests# _test_import_traces()

---

## 🧩 Part II: Framework Extension Tools

Tools for integrating with and extending the SpiralSafe ecosystem frameworks.

### 2.1 Ecosystem Repository Discovery

Tools for discovering and working with SpiralSafe ecosystem repositories.

In [ ]:
@dataclass
class EcosystemRepo:
    """Represents a SpiralSafe ecosystem repository."""
    name: str
    github_url: str
    purpose: str
    version: str
    local_path: Optional[str] = None
    is_available: bool = False


# Define the SpiralSafe ecosystem
ECOSYSTEM_REPOS = [
    EcosystemRepo(
        name="SpiralSafe",
        github_url="https://github.com/toolate28/SpiralSafe",
        purpose="Documentation hub, coherence engine core",
        version="v2.1.0"
    ),
    EcosystemRepo(
        name="wave-toolkit",
        github_url="https://github.com/toolate28/wave-toolkit",
        purpose="Coherence detection tools",
        version="v1.0.0"
    ),
    EcosystemRepo(
        name="ClaudeNPC-Server-Suite",
        github_url="https://github.com/toolate28/ClaudeNPC-Server-Suite",
        purpose="AI NPCs for Minecraft (HOPE NPCs)",
        version="v2.1.0"
    ),
    EcosystemRepo(
        name="kenl",
        github_url="https://github.com/toolate28/kenl",
        purpose="Infrastructure-aware AI orchestration",
        version="v1.0.0"
    ),
    EcosystemRepo(
        name="quantum-redstone",
        github_url="https://github.com/toolate28/quantum-redstone",
        purpose="Quantum computing education via Redstone",
        version="Available"
    )
]


def discover_local_repos(search_paths: Optional[List[str]] = None) -> List[EcosystemRepo]:
    """
    Discover locally available ecosystem repositories.
    
    Args:
        search_paths: List of paths to search for repos
        
    Returns:
        Updated list of EcosystemRepo objects with local paths
    """
    if search_paths is None:
        home = Path.home()
        search_paths = [
            str(home),
            str(home / "repos"),
            str(home / "projects"),
            str(Path.cwd()),
            str(Path.cwd().parent)
        ]
    
    repos = []
    for repo in ECOSYSTEM_REPOS:
        repo_copy = EcosystemRepo(
            name=repo.name,
            github_url=repo.github_url,
            purpose=repo.purpose,
            version=repo.version
        )
        
        # Search for local copy
        for search_path in search_paths:
            potential_path = Path(search_path) / repo.name
            if potential_path.exists() and (potential_path / ".git").exists():
                repo_copy.local_path = str(potential_path)
                repo_copy.is_available = True
                break
        
        repos.append(repo_copy)
    
    return repos


def display_ecosystem_status():
    """Display the status of ecosystem repositories."""
    repos = discover_local_repos()
    
    print("🌀 SpiralSafe Ecosystem Status")
    print("=" * 60)
    
    for repo in repos:
        status = "✅ Local" if repo.is_available else "📡 Remote only"
        print(f"\n📦 {repo.name} ({repo.version})")
        print(f"   Purpose: {repo.purpose}")
        print(f"   Status: {status}")
        if repo.local_path:
            print(f"   Path: {repo.local_path}")
        print(f"   GitHub: {repo.github_url}")


# Display ecosystem status
display_ecosystem_status()

### 2.2 Framework Analysis Tools

Tools for analyzing and understanding existing framework structures.

In [ ]:
@dataclass
class FrameworkAnalysis:
    """Analysis results for a framework/repository."""
    path: str
    name: str
    file_count: int
    languages: Dict[str, int]
    structure: Dict[str, List[str]]
    has_tests: bool
    has_docs: bool
    has_ai_agents_config: bool


def analyze_framework(repo_path: str) -> Optional[FrameworkAnalysis]:
    """
    Analyze a framework/repository structure.
    
    Args:
        repo_path: Path to the repository
        
    Returns:
        FrameworkAnalysis object or None if path doesn't exist
    """
    path = Path(repo_path)
    if not path.exists():
        return None
    
    # Language extensions mapping
    lang_extensions = {
        ".py": "Python",
        ".ps1": "PowerShell",
        ".psm1": "PowerShell",
        ".js": "JavaScript",
        ".ts": "TypeScript",
        ".json": "JSON",
        ".md": "Markdown",
        ".yaml": "YAML",
        ".yml": "YAML",
        ".sh": "Shell"
    }
    
    languages: Dict[str, int] = {}
    structure: Dict[str, List[str]] = {}
    file_count = 0
    
    # Walk through repository
    for item in path.rglob("*"):
        # Skip hidden directories and common ignore patterns
        if any(part.startswith(".") for part in item.parts):
            continue
        if "node_modules" in item.parts or "__pycache__" in item.parts:
            continue
        
        if item.is_file():
            file_count += 1
            ext = item.suffix.lower()
            if ext in lang_extensions:
                lang = lang_extensions[ext]
                languages[lang] = languages.get(lang, 0) + 1
            
            # Build structure
            rel_path = item.relative_to(path)
            if len(rel_path.parts) > 1:
                dir_name = str(rel_path.parts[0])
                if dir_name not in structure:
                    structure[dir_name] = []
                if len(structure[dir_name]) < 5:  # Limit entries
                    structure[dir_name].append(item.name)
    
    return FrameworkAnalysis(
        path=str(path),
        name=path.name,
        file_count=file_count,
        languages=languages,
        structure=structure,
        has_tests=(path / "tests").exists() or (path / "test").exists(),
        has_docs=(path / "docs").exists() or (path / "README.md").exists(),
        has_ai_agents_config=(path / "AI_AGENTS.md").exists()
    )


def display_framework_analysis(analysis: FrameworkAnalysis):
    """Display framework analysis results."""
    print(f"📊 Framework Analysis: {analysis.name}")
    print("=" * 50)
    print(f"Path: {analysis.path}")
    print(f"Total Files: {analysis.file_count}")
    
    print("\n📝 Languages:")
    for lang, count in sorted(analysis.languages.items(), key=lambda x: -x[1]):
        print(f"   {lang}: {count} files")
    
    print("\n📁 Structure:")
    for dir_name, files in analysis.structure.items():
        print(f"   {dir_name}/")
        for f in files[:3]:
            print(f"      └── {f}")
        if len(files) > 3:
            print(f"      └── ... ({len(files) - 3} more)")
    
    print("\n✅ Features:")
    print(f"   Tests: {'Yes' if analysis.has_tests else 'No'}")
    print(f"   Docs: {'Yes' if analysis.has_docs else 'No'}")
    print(f"   AI Agents Config: {'Yes' if analysis.has_ai_agents_config else 'No'}")


# Analyze current repository
current_repo = Path.cwd()
analysis = analyze_framework(str(current_repo))
if analysis:
    display_framework_analysis(analysis)

### 2.3 Log Collection Integration

Python implementation of log collection - extends `Wave.Logging.psm1` functionality.

In [ ]:
@dataclass
class LogSource:
    """Represents a log source in the ecosystem."""
    path: str
    source: str
    repo: str
    kind: str  # 'csv', 'text', 'json'
    exists: bool = False


def discover_log_sources() -> List[LogSource]:
    """
    Discover log sources across the SpiralSafe ecosystem.
    
    Returns:
        List of LogSource objects
    """
    home = Path.home()
    sources = []
    
    # Gaming logs
    bf6_log = home / "bf6_performance_log.csv"
    sources.append(LogSource(
        path=str(bf6_log),
        source="system",
        repo="gaming",
        kind="csv",
        exists=bf6_log.exists()
    ))
    
    # SpiralSafe bridges
    bridge_dir = home / "SpiralSafe-FromGitHub" / "bridges"
    if bridge_dir.exists():
        for log_file in bridge_dir.rglob("*.log"):
            sources.append(LogSource(
                path=str(log_file),
                source="spiralsafe",
                repo="SpiralSafe",
                kind="text",
                exists=True
            ))
    
    # Quantum-redstone logs
    qr_dir = home / "quantum-redstone"
    if qr_dir.exists():
        for log_file in qr_dir.rglob("*.log"):
            sources.append(LogSource(
                path=str(log_file),
                source="quantum",
                repo="quantum-redstone",
                kind="text",
                exists=True
            ))
    
    # ClaudeNPC logs
    cnpc_dir = home / "repos" / "ClaudeNPC-Server-Suite"
    if cnpc_dir.exists():
        for log_file in cnpc_dir.rglob("*.log"):
            sources.append(LogSource(
                path=str(log_file),
                source="claudenpc",
                repo="ClaudeNPC-Server-Suite",
                kind="text",
                exists=True
            ))
    
    # ATOM trail
    atom_trail = home / ".atom-trail"
    sources.append(LogSource(
        path=str(atom_trail),
        source="spiralsafe",
        repo="SpiralSafe",
        kind="text",
        exists=atom_trail.exists()
    ))
    
    return sources


def display_log_sources():
    """Display discovered log sources."""
    sources = discover_log_sources()
    available = [s for s in sources if s.exists]
    
    print("📋 Log Sources Discovery")
    print("=" * 50)
    print(f"Found {len(available)} available log sources")
    
    if available:
        print("\n✅ Available:")
        for src in available:
            print(f"   [{src.source}] {src.repo} - {Path(src.path).name}")
    
    missing = [s for s in sources if not s.exists]
    if missing:
        print("\n⚪ Not Found (expected locations):")
        for src in missing[:5]:  # Limit display
            print(f"   [{src.source}] {src.repo} - {Path(src.path).name}")


# Display log sources
display_log_sources()

---

## 🚀 Part III: Interactive Tooling

Interactive tools for working with Wave and the ecosystem.

### 3.1 Quick Context Export

Export current context for use in other tools or AI sessions.

In [ ]:
def export_context(output_path: Optional[str] = None) -> str:
    """
    Export the current Wave context to a JSON file.
    
    Args:
        output_path: Custom output path (default: .claude/wave_context.json)
        
    Returns:
        Path to the exported file
    """
    if output_path is None:
        output_path = ".claude/wave_context.json"
    
    output = Path(output_path)
    output.parent.mkdir(parents=True, exist_ok=True)
    
    ctx = get_wave_context()
    
    with open(output, "w", encoding="utf-8") as f:
        f.write(ctx.to_json())
    
    print(f"✅ Context exported to: {output}")
    return str(output)


def export_prompt(output_path: Optional[str] = None) -> str:
    """
    Export the system prompt to a markdown file.
    
    Args:
        output_path: Custom output path (default: .claude/prompts/wave-system.md)
        
    Returns:
        Path to the exported file
    """
    if output_path is None:
        output_path = ".claude/prompts/wave-system.md"
    
    output = Path(output_path)
    output.parent.mkdir(parents=True, exist_ok=True)
    
    ctx = get_wave_context()
    prompt = generate_system_prompt(ctx)
    
    with open(output, "w", encoding="utf-8") as f:
        f.write(prompt)
    
    print(f"✅ System prompt exported to: {output}")
    return str(output)


# Example: Export both context and prompt
print("📤 Exporting Wave artifacts...")
# Uncomment to actually export:
# export_context()
# export_prompt()
print("(Uncomment the lines above to export files)")

### 3.2 Framework Extension Helper

Tools for extending existing frameworks with Wave patterns.

In [ ]:
def generate_ai_agents_template(repo_name: str, structure: Dict[str, List[str]]) -> str:
    """
    Generate an AI_AGENTS.md template for a repository.
    
    Args:
        repo_name: Name of the repository
        structure: Directory structure dictionary
        
    Returns:
        Markdown template string
    """
    dirs_list = "\n".join([f"├── {d}/" for d in structure.keys()])
    
    template = f"""# AI Agent Coordination Rules

*This file tells ALL AI agents (Claude, Ollama, GPT, etc.) where to put things in {repo_name}.*

[![SpiralSafe](https://img.shields.io/badge/🌀_SpiralSafe-Ecosystem-purple?style=flat-square)](https://github.com/toolate28/SpiralSafe)
[![Wave Toolkit](https://img.shields.io/badge/🌊_Wave_Toolkit-Blueprint-0066FF?style=flat-square)](https://github.com/toolate28/wave-toolkit)

> **Part of the [SpiralSafe Ecosystem](https://github.com/toolate28/SpiralSafe)**

---

## The Golden Rule

**Follow the established directory structure. Never create loose files in root.**

---

## Directory Structure

```text
{repo_name}/
{dirs_list}
```

---

## Placement Rules

| Type | Location | Example |
|------|----------|---------||
| Documentation | `docs/` | `*.md` |
| Tests | `tests/` | `*_test.py`, `*.Tests.ps1` |
| Configuration | root or `config/` | `*.json`, `*.yaml` |

---

*This file is the source of truth for AI agent behavior in this workspace.*
"""
    return template


def generate_ecosystem_badge_header(repo_name: str) -> str:
    """
    Generate ecosystem badge header for documentation files.
    
    Args:
        repo_name: Name of the repository
        
    Returns:
        Markdown badge header string
    """
    return f"""[![SpiralSafe](https://img.shields.io/badge/🌀_SpiralSafe-Ecosystem-purple?style=flat-square)](https://github.com/toolate28/SpiralSafe)
[![{repo_name}](https://img.shields.io/badge/📦_{repo_name}-Main-blue?style=flat-square)](README.md)

> **Part of the [SpiralSafe Ecosystem](https://github.com/toolate28/SpiralSafe)**
"""


# Display example output
print("📝 Example: Ecosystem Badge Header")
print("=" * 40)
print(generate_ecosystem_badge_header("example-repo"))

### 3.3 Session Summary Generator

Generate summaries of Wave sessions for documentation and review.

In [ ]:
def generate_session_summary(session: WaveSession) -> str:
    """
    Generate a markdown summary of a Wave session.
    
    Args:
        session: WaveSession object
        
    Returns:
        Markdown summary string
    """
    ctx = session.context
    
    summary = f"""# Wave Session Summary

## Session Info
- **ID:** {session.session_id}
- **Timestamp:** {session.timestamp}
- **Task:** {session.task or 'Not specified'}

## Environment
- **Machine:** {ctx.machine.name} ({ctx.machine.arch})
- **OS:** {ctx.machine.os}
- **Working Directory:** {ctx.session.cwd}
- **Git Repo:** {'Yes' if ctx.session.is_git_repo else 'No'}

## Log Entries
"""
    
    for entry in session.log_entries:
        summary += f"- [{entry['timestamp']}] **{entry['type']}**: {entry['content']}\n"
    
    summary += "\n---\n*Generated by Wave Toolkit Project Book*\n"
    
    return summary


# Generate and display summary for demo session
demo_session.add_log("action", "Explored repository structure")
demo_session.add_log("action", "Reviewed ecosystem integration")

summary = generate_session_summary(demo_session)
print(summary)

---

## 📚 Part IV: Reference & Documentation

Quick reference for Wave patterns and ecosystem integration.

### 4.1 Communication Patterns Reference

Key patterns from the Wave communication philosophy.

In [ ]:
COMMUNICATION_PATTERNS = {
    "your_lead": {
        "pattern": "'Your lead, do what you feel is best'",
        "description": "Gives AI full creative latitude while signaling trust",
        "when_to_use": "When the goal is clear but the path isn't"
    },
    "why_before_what": {
        "pattern": "Share the 'why' before the 'what'",
        "description": "Context shapes every decision",
        "when_to_use": "Always - explain purpose before requesting action"
    },
    "ultrathink": {
        "pattern": "'Ultrathink' / 'God-mode' signals",
        "description": "Explicit permission to go deep, think thoroughly",
        "when_to_use": "Complex architectural decisions, important work"
    },
    "parallel_trust": {
        "pattern": "Parallel trust",
        "description": "Letting AI run multiple operations simultaneously",
        "when_to_use": "Multi-task scenarios, maximize throughput"
    },
    "outcome_over_process": {
        "pattern": "Outcome over process",
        "description": "State the real goal, not the task",
        "when_to_use": "When task isn't the real goal"
    }
}


def display_patterns():
    """Display communication patterns reference."""
    print("💬 Communication Patterns Reference")
    print("=" * 50)
    
    for key, pattern in COMMUNICATION_PATTERNS.items():
        print(f"\n🔹 {pattern['pattern']}")
        print(f"   {pattern['description']}")
        print(f"   📌 When: {pattern['when_to_use']}")


display_patterns()

### 4.2 Ecosystem Quick Links

Quick access to ecosystem documentation and resources.

In [ ]:
ECOSYSTEM_LINKS = {
    "SpiralSafe Hub": "https://github.com/toolate28/SpiralSafe",
    "Wave Toolkit": "https://github.com/toolate28/wave-toolkit",
    "HOPE NPCs": "https://github.com/toolate28/ClaudeNPC-Server-Suite",
    "kenl": "https://github.com/toolate28/kenl",
    "Quantum Redstone": "https://github.com/toolate28/quantum-redstone",
    "Contributing Guide": "https://github.com/toolate28/SpiralSafe/blob/main/CONTRIBUTING.md",
    "Portfolio": "https://github.com/toolate28/SpiralSafe/blob/main/PORTFOLIO.md"
}


def display_ecosystem_links():
    """Display ecosystem quick links."""
    print("🔗 Ecosystem Quick Links")
    print("=" * 50)
    
    for name, url in ECOSYSTEM_LINKS.items():
        print(f"   📌 {name}")
        print(f"      {url}")


display_ecosystem_links()

---

## 🎯 Quick Start Examples

Common use cases and quick examples.

In [ ]:
# Example 1: Full context capture and prompt generation
def quick_start_demo():
    """Demonstrate the complete Wave workflow."""
    print("🌊 Wave Toolkit Quick Start Demo")
    print("=" * 50)
    
    # Step 1: Capture context
    print("\n[1/4] Capturing context...")
    ctx = get_wave_context()
    print(f"      ✅ Context captured at {ctx.timestamp}")
    
    # Step 2: Generate prompt
    print("\n[2/4] Generating system prompt...")
    prompt = generate_system_prompt(ctx)
    print(f"      ✅ Prompt generated ({len(prompt)} chars)")
    
    # Step 3: Create session
    print("\n[3/4] Creating session...")
    session = create_wave_session("Quick Start Demo")
    print(f"      ✅ Session: {session.session_id}")
    
    # Step 4: Analyze ecosystem
    print("\n[4/4] Analyzing ecosystem...")
    repos = discover_local_repos()
    available = sum(1 for r in repos if r.is_available)
    print(f"      ✅ Found {available}/{len(repos)} local repos")
    
    print("\n🎉 Demo complete!")
    return session


# Run the demo
demo_result = quick_start_demo()

---

## Attribution

This work emerges from **Hope&&Sauced** collaboration—human-AI partnership where both contributions are substantive and neither party could have produced the result alone.

See the [SpiralSafe PORTFOLIO.md](https://github.com/toolate28/SpiralSafe/blob/main/PORTFOLIO.md) for complete ecosystem showcase.

---

*~ Hope&&Sauced*

✦ *The Evenstar Guides Us* ✦